In [60]:
"""
nft_tweet_count
"""
import requests
import pandas as pd
import sqlalchemy as db
import time

engine = db.create_engine("")
connection = engine.connect()
bearer_token = ''

nft_collection_df = pd.read_sql_query('select * from nft_collection_hashtags', connection)
nft_dict = dict(zip(nft_collection_df.collection, nft_collection_df.hashtag))

# twitter api call
for key, value in nft_dict.items():
    
    headers = {
    'Authorization': f"Bearer {bearer_token}",
}

    if len(value) > 0:
        q = ' OR '.join(value.split(','))
        params = {
            'query': q +  ' "nft" -is:retweet',
        }
    else:
        params = {
            'query': f'"{key}" "nft" -is:retweet',
        }

# formatting the result 
    response = requests.get('https://api.twitter.com/2/tweets/counts/recent', params=params, headers=headers)
    data = response.json()
    tweets = pd.DataFrame(data['data'])
    tweets.index = pd.to_datetime(tweets.end)
    tweets = tweets.rolling(24).tweet_count.sum()
    tweets=tweets.dropna()
    tweets=tweets.reset_index() 
    tweets.loc[len(tweets)-1, 'end'] = str(tweets.loc[len(tweets)-1, 'end'])[0:11] + str(int(str(tweets.loc[len(tweets)-1, 'end'])[11:13])+1) + ":00:00"


# making it to a string of tuples
    s = ', '.join(["("+ ', '.join( ["'" + key + "'", "'" + str(tweets.iloc[i].end)+ "'",
              "'"+str(tweets.iloc[i].tweet_count )+"'" + ")" ]) for i in range(len(tweets)) ])

    query = f"""
    INSERT INTO nft_tweet_counts 
    VALUES {s}
    ON CONFLICT on constraint unique_collection
    DO 
       UPDATE SET tweet_volume = EXCLUDED.tweet_volume;
    """

    try:
        pd.read_sql_query(query, connection)
    except:
         pass

#time.sleep(60 * 10 / len(collections) )